In [1]:
import os
import pandas as pd
import numpy as np

#import google cloud library
from google.cloud import bigquery
from google.cloud import storage
from google.cloud import aiplatform
from support_functions import missing_value, fill_missing, list_dtypes

## sklearn module
from sklearn.linear_model import LogisticRegression 
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.preprocessing import LabelEncoder
import pickle

In [2]:
## set up authentication using services account 
# Authenticate using service account
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = "../arif-014/sa-development.json"
# os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = "../vertex-ai-ml/dti-ds-31329ac0651d.json"
 # Load data from Google Biquery to local machine
project_id = 'dti-ds'
dataset_id = 'arif_dataset_014'
table_id = 'data_customer_lifetime_value'
region = 'us-central1'
bucket_name = 'arif_gcs_014'
blob_name = 'data/data_customer_lifetime_value.csv'

In [3]:
from sklearn import show_versions
show_versions()


System:
    python: 3.10.14 (main, May  6 2024, 19:42:50) [GCC 11.2.0]
executable: /home/ariefhadiyan/miniconda3/envs/clv-env/bin/python
   machine: Linux-6.1.85+-x86_64-with-glibc2.35

Python dependencies:
      sklearn: 1.5.1
          pip: 24.0
   setuptools: 65.6.3
        numpy: 1.23.5
        scipy: 1.10.0
       Cython: 3.0.10
       pandas: 2.2.2
   matplotlib: 3.7.0
       joblib: 1.4.2
threadpoolctl: 3.5.0

Built with OpenMP: True

threadpoolctl info:
       user_api: blas
   internal_api: openblas
    num_threads: 4
         prefix: libopenblas
       filepath: /home/ariefhadiyan/miniconda3/envs/clv-env/lib/python3.10/site-packages/numpy.libs/libopenblas64_p-r0-742d56dc.3.20.so
        version: 0.3.20
threading_layer: pthreads
   architecture: Haswell

       user_api: blas
   internal_api: openblas
    num_threads: 4
         prefix: libopenblas
       filepath: /home/ariefhadiyan/miniconda3/envs/clv-env/lib/python3.10/site-packages/scipy.libs/libopenblasp-r0-41284840.3.18

In [4]:
# Read the model from Google Cloud Storage
try : 
    storage_client = storage.Client(project=project_id)
    bucket = storage_client.get_bucket(bucket_name) # Add bucket name
    blob_model = bucket.blob('model/Model_Customer_Lifetime_Value_Prediction_RF_arif_014.pkl')
    blob_model.download_to_filename('model2.pkl')

    print ("Read model succeeded")
except:
    raise TypeError("An exception occurred")

Read model succeeded


In [5]:
pip install category_encoders

  Using cached category_encoders-2.6.3-py2.py3-none-any.whl.metadata (8.0 kB)
  Using cached statsmodels-0.14.2-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (9.2 kB)
  Using cached patsy-0.5.6-py2.py3-none-any.whl.metadata (3.5 kB)
Using cached category_encoders-2.6.3-py2.py3-none-any.whl (81 kB)
Using cached patsy-0.5.6-py2.py3-none-any.whl (233 kB)
Using cached statsmodels-0.14.2-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (10.8 MB)
Note: you may need to restart the kernel to use updated packages.


In [6]:
# Load the model
with open('model2.pkl', 'rb') as f:
    loaded_model = pickle.load(f)

In [7]:
tester = pd.DataFrame({
    'Vehicle Class':['Four-Door Car'],
    'Coverage':['Basic'],
    'Renew Offer Type':['Offer1'],
    'Employment Status':['Employed'],
    'Marital Status':['Married'],
    'Education':['Bachelor'],
    'Number of Policies':[5],
    'Monthly Premium Auto':[500],
    'Total Claim Amount':[1000],
    'Income':[80000]
})



In [8]:
loaded_model.predict(tester)

array([15579.40197623])

In [9]:
# load data from BQ
from google.cloud import bigquery
client = bigquery.Client(project=project_id)

# query
query_job = client.query(f"""select * from {dataset_id}.{table_id}""")
auto_cloud = query_job.result().to_dataframe()

auto_cloud.columns = tester.columns

auto_cloud

/home/ariefhadiyan/miniconda3/envs/clv-env/lib/python3.10/site-packages/google/cloud/bigquery/table.py:1727: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(


,Vehicle Class,Coverage,Renew Offer Type,Employment Status,Marital Status,Education,Number of Policies,Monthly Premium Auto,Total Claim Amount,Income
0,Luxury SUV,Premium,Offer1,Retired,Divorced,College,8,256,1228.800000,26000
1,Luxury SUV,Premium,Offer1,Retired,Married,Bachelor,1,271,1300.800000,14290
2,Luxury SUV,Premium,Offer1,Retired,Married,Bachelor,1,271,1300.800000,14290
3,Luxury SUV,Premium,Offer1,Retired,Married,Bachelor,1,271,1300.800000,14290
4,Luxury SUV,Premium,Offer1,Retired,Married,Bachelor,1,271,1300.800000,14290
...,...,...,...,...,...,...,...,...,...,...
5664,Luxury SUV,Basic,Offer1,Medical Leave,Married,Bachelor,3,194,931.200000,29790
5665,Luxury SUV,Basic,Offer1,Medical Leave,Divorced,Bachelor,2,198,950.400000,17130
5666,Luxury SUV,Extended,Offer1,Medical Leave,Married,High School or Below,3,206,1254.137899,21921
5667,Luxury SUV,Extended,Offer3,Medical Leave,Divorced,Doctor,2,240,1152.000000,21275


In [10]:
# load the model
with open('model2.pkl', 'rb') as f:
    loaded_model = pickle.load(f)

y_pred_cloud_new_data = loaded_model.predict(tester)
y_pred_cloud_new_data

array([15579.40197623])

In [11]:
new_test_cloud = tester.copy()
new_test_cloud['CLV Prediction'] = y_pred_cloud_new_data
new_test_cloud

,Vehicle Class,Coverage,Renew Offer Type,Employment Status,Marital Status,Education,Number of Policies,Monthly Premium Auto,Total Claim Amount,Income,CLV Prediction
0,Four-Door Car,Basic,Offer1,Employed,Married,Bachelor,5,500,1000,80000,15579.401976


In [12]:
y_pred_file_cloud = loaded_model.predict(auto_cloud)
y_pred_file_cloud[:13]

array([14875.7179533 ,  8913.91681598,  8913.91681598,  8913.91681598,
        8913.91681598,  8913.91681598,  2349.56569852,  2349.56569852,
        2349.56569852,  2349.56569852,  2349.56569852,  9211.02759744,
        9211.02759744])

In [13]:
auto_cloud['CLV Prediction'] = y_pred_file_cloud
auto_cloud

,Vehicle Class,Coverage,Renew Offer Type,Employment Status,Marital Status,Education,Number of Policies,Monthly Premium Auto,Total Claim Amount,Income,CLV Prediction
0,Luxury SUV,Premium,Offer1,Retired,Divorced,College,8,256,1228.800000,26000,14875.717953
1,Luxury SUV,Premium,Offer1,Retired,Married,Bachelor,1,271,1300.800000,14290,8913.916816
2,Luxury SUV,Premium,Offer1,Retired,Married,Bachelor,1,271,1300.800000,14290,8913.916816
3,Luxury SUV,Premium,Offer1,Retired,Married,Bachelor,1,271,1300.800000,14290,8913.916816
4,Luxury SUV,Premium,Offer1,Retired,Married,Bachelor,1,271,1300.800000,14290,8913.916816
...,...,...,...,...,...,...,...,...,...,...,...
5664,Luxury SUV,Basic,Offer1,Medical Leave,Married,Bachelor,3,194,931.200000,29790,15248.550555
5665,Luxury SUV,Basic,Offer1,Medical Leave,Divorced,Bachelor,2,198,950.400000,17130,15475.072273
5666,Luxury SUV,Extended,Offer1,Medical Leave,Married,High School or Below,3,206,1254.137899,21921,15465.455141
5667,Luxury SUV,Extended,Offer3,Medical Leave,Divorced,Doctor,2,240,1152.000000,21275,15525.342872


In [14]:
# Construct a BigQuery client object
from google.cloud import bigquery

table_id = 'unseen_car_clv_predicted'

client = bigquery.Client()

#Define the full table ID
table_full_id = f"{client.project}.{dataset_id}.{table_id}"

auto_cloud.columns = ['_'.join(i.split(' ')) for i in auto_cloud.columns]

#Load the Dataframe into the BigQuery table
job = client.load_table_from_dataframe(auto_cloud, table_full_id)

# Wait for the job to complete
job.result()
print(f"Loaded {job.output_rows} rows into {table_full_id}")

Loaded 5669 rows into dti-ds.arif_dataset_014.unseen_car_clv_predicted
